In [ ]:
import numpy as np
from transformers import PreTrainedTokenizerFast
import torch
import pandas as pd

In [ ]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join('..', '..')))
from Functions_generation import generate_a_song_structure, sample_with_temp_topk, Subword_LSTM

In [ ]:
tokenizer = PreTrainedTokenizerFast(
    tokenizer_file="../../../Corpus/Encoding_RNN_LSTM/Subword/rap_tokenizer.json",
    pad_token = "<PAD>")

In [ ]:
matrix = pd.read_csv("../../../Markov/transition_matrix.csv")

states = np.array(matrix.iloc[6])
prob_transi = np.array(matrix.iloc[0:6])

In [ ]:
vocab_size = tokenizer.vocab_size
model = Subword_LSTM(vocab_size)   

In [ ]:
ckpt = torch.load("../Models/LSTM_model.pt", map_location="cpu")

new_state_dict = {}
for key, value in ckpt["model_state_dict"].items():
    new_key = key.replace("_orig_mod.", "")           
    new_state_dict[new_key] = value

ckpt["model_state_dict"] = new_state_dict

model.load_state_dict(ckpt["model_state_dict"])

In [ ]:
struct = generate_a_song_structure(prob_transi.astype(float),states)

#struct = ['<BEGINNING>', '<COUPLET>', '<REFRAIN>', '<COUPLET>', '<REFRAIN>', '<END>']
struct = ['α', 'γ', 'ε', 'γ', 'ε', 'θ']

Forbidden to generate

In [ ]:
forbidden_gen = tokenizer.encode("".join(['α', 'β', 'γ', 'ε', 'ζ', 'η', 'θ']))

vocab = tokenizer.get_vocab()  
tokens_with_slash = [tokenizer.encode(tok)[0] for tok in vocab.keys() if "/" in tok]

In [ ]:
decod_structure = {"β" : "<INTRO>",
                   "γ" : "<COUPLET>",
                   "ε" : "<REFRAIN>",
                   "ζ" : "<PONT>",
                   "η" : "<OUTRO>",
                   "θ" : "<END>"}

In [ ]:
tokenizer.encode(" ")

In [ ]:
tokenizer.decode(0)

In [ ]:
context_gen = "aujourd'hui nous faisons"
context_encode = tokenizer.encode(context_gen)
context_size = len(context_encode)

hid = model.init_hidden(batch_size=1, device="cpu")

for i in range(len(struct) - 2):

    print()
    print(decod_structure[struct[i+1]])

    if i == 0 : 
        context_encode.insert(0,tokenizer.encode(struct[i+1])[0])
        context_encode.insert(1,tokenizer.encode("\n")[0])
        encoded = torch.tensor(context_encode, dtype=torch.long).unsqueeze(0)

        print("".join([tokenizer.decode(context_encode[1:])]), end="")

        out, hid = model(encoded, hid)
        next_input = encoded[:, -1].unsqueeze(0)  # shape (1, 1)

    else : 
        next_input = torch.tensor(tokenizer.encode(struct[i+1])[0],dtype=torch.long).view(1,1)

    last_input = 0
    first_input = True

    while last_input not in tokens_with_slash:
        with torch.no_grad():
            out, hid = model(next_input, hid)   # out shape: (1, 1, vocab_size)

            out[0, -1][forbidden_gen] = float("-inf")

        # 4. On échantillonne un token
            next_token = sample_with_temp_topk(out[0, -1], temperature=0.5, top_k=20)
            last_input = next_token.item()

        # 5. Conditions d’impression
            if first_input and last_input in tokens_with_slash:  # Si début et le modèle sort directement <END>
                last_input = 0
                continue

            elif next_token.item() != tokens_with_slash:
                first_input = False
                print(tokenizer.decode(next_token.item()), end="")

        # 6. Préparer la prochaine entrée
            next_input = next_token.view(1, 1)

print("\n", decod_structure[struct[-1]])
